# Set up

In [78]:
import os
import ee
import geemap
import math

In [82]:
ee.Initialize()

In [79]:
period_of_interest = ee.Filter.date('2017-01-01', '2019-12-31')
inBands = ["B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12"]
outBands = inBands.append(["NDVI","NDVI_STD","BSI"])
Area_Nice = ee.Geometry.Polygon(
        [
            [
                [6.560201, 43.526958],
                [6.560201, 44.371294],
                [7.51888, 44.371294],
                [7.51888, 43.526958],
                [6.560201, 43.526958],
            ]
        ]
    )

In [80]:
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10 # 1024
    cirrusBitMask = 1 << 11 # 2048

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)
    
    helper = image.updateMask(mask).divide(10000)
    ee.Image(helper.copyProperties(image, properties=["system:time_start"]))
    ee.Image(helper.copyProperties(image, properties=["CLOUD_COVERAGE_PERCENTAGE"]))

    return helper

In [84]:
s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR') 
    .filterBounds(Area_Nice)
    .filter(period_of_interest)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) 
    .map(maskS2clouds)
    .select(["B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12"])# cannot use variable inBands!!
)
median = s2.median()

visParams = {
    'bands': ['B4', 'B3', 'B2'],#'B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.3,
}

Map = geemap.Map()
Map.centerObject(Area_Nice)
Map.addLayer(median, visParams,'median')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [72]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [73]:
S2_NDVI = s2.map(addNDVI)

In [74]:
recent_S2 = ee.Image(S2_NDVI.sort('system:time_start', False).first())

In [75]:
print('Recent image: ', recent_S2)

Recent image:  ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.limit",
          "arguments": {
            "ascending": {
              "constantValue": false
            },
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.addBands",
                          "arguments": {
                            "dstImg": {
                              "argumentReference": "_MAPPING_VAR_0_0"
                            },
 

In [76]:
NDVIpalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']

In [77]:
Map.addLayer(recent_S2.select('NDVI'), {'palette': NDVIpalette}, 'Recent Sentinel NDVI')
Map.centerObject(Area_Nice)
Map

Map(center=[43.94962715775877, 7.0395405], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [85]:
ten_day_S2 = S2_NDVI.sort('system:time_start', False).limit (10)
# print('Recent images: ', ten_day_S2)
Map.addLayer(ten_day_S2.select('NDVI').median(), {'palette': NDVIpalette}, '10 Day Median Sentinel NDVI')
Map

Map(center=[43.94962715775877, 7.0395405], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

# Chart

In [48]:
import pandas as pd
import altair as alt
import numpy as np
import folium

ModuleNotFoundError: No module named 'altair'

In [49]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-20, 'years'), today)

In [53]:
# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [58]:
ndvi = ee.ImageCollection('MODIS/006/MOD13A2').filterDate(date_range).select('NDVI').filterBounds(Area_Nice)

ndvi_dict = fc_to_dict(ndvi).getInfo()
ndvi_df = pd.DataFrame(ndvi_dict)
display(ndvi_df)
print(ndvi_df.dtypes)

,google:max_source_file_timestamp,system:asset_size,system:band_names,system:bands,system:footprint,system:id,system:index,system:time_end,system:time_start,system:version
0,1528309196580,2792195822,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_07_28,2002_07_28,1029196800000,1027814400000,1.638765e+15
1,1528309227932,2779335570,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_08_13,2002_08_13,1030579200000,1029196800000,1.638765e+15
2,1528310673716,2781385101,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_08_29,2002_08_29,1031961600000,1030579200000,1.638765e+15
3,1528309312684,2775264985,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_09_14,2002_09_14,1033344000000,1031961600000,1.638765e+15
4,1528309218153,2801753874,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_09_30,2002_09_30,1034726400000,1033344000000,1.638765e+15
...,...,...,...,...,...,...,...,...,...,...
454,1652526019000,2850848972,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2022_04_23,2022_04_23,1652054400000,1650672000000,1.652565e+15
455,1653558187000,2838197115,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2022_05_09,2022_05_09,1653436800000,1652054400000,1.653614e+15
456,1655022813000,2823228158,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2022_05_25,2022_05_25,1654819200000,1653436800000,1.655330e+15
457,1656548348000,2802598825,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2022_06_10,2022_06_10,1656201600000,1654819200000,1.656770e+15


google:max_source_file_timestamp      int64
system:asset_size                     int64
system:band_names                    object
system:bands                         object
system:footprint                     object
system:id                            object
system:index                         object
system:time_end                       int64
system:time_start                     int64
system:version                      float64
dtype: object


In [57]:
ndvi_df.head(5)

,google:max_source_file_timestamp,system:asset_size,system:band_names,system:bands,system:footprint,system:id,system:index,system:time_end,system:time_start,system:version
0,1528309196580,2792195822,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_07_28,2002_07_28,1029196800000,1027814400000,1.638765e+15
1,1528309227932,2779335570,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_08_13,2002_08_13,1030579200000,1029196800000,1.638765e+15
2,1528310673716,2781385101,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_08_29,2002_08_29,1031961600000,1030579200000,1.638765e+15
3,1528309312684,2775264985,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_09_14,2002_09_14,1033344000000,1031961600000,1.638765e+15
4,1528309218153,2801753874,[NDVI],"{'NDVI': {'data_type': {'type': 'PixelType', '...","{'type': 'LinearRing', 'coordinates': [[-180, ...",MODIS/006/MOD13A2/2002_09_30,2002_09_30,1034726400000,1033344000000,1.638765e+15


# chart

In [64]:
import geemap.chart as chart

In [68]:
Map = geemap.Map()

features = ee.FeatureCollection('projects/google/charts_feature_example').select(
    '[0-9][0-9]_tmean|label'
)

Map.addLayer(features, {}, "Ecoregions")
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [60]:
df = geemap.ee_to_pandas(features)
df

,01_tmean,07_tmean,08_tmean,11_tmean,05_tmean,label,09_tmean,10_tmean,04_tmean,03_tmean,12_tmean,06_tmean,02_tmean
0,5.791036,25.066577,23.845259,9.849321,19.032815,Desert,21.454795,15.997988,14.251640,10.454658,5.641386,23.790506,7.645011
1,2.792467,17.850177,17.919983,4.784706,10.395376,Forest,15.206573,10.081709,7.120137,5.032932,2.317886,13.776134,3.609074
2,-3.756608,22.753059,21.848346,2.021918,13.643875,Grassland,16.401770,9.263021,7.721306,2.570146,-3.426706,19.033558,-1.990220


In [61]:
xProperty = "label"
yProperties = [str(x).zfill(2) + "_tmean" for x in range(1, 13)]

labels = [
    'Jan',
    'Feb',
    'Mar',
    'Apr',
    'May',
    'Jun',
    'Jul',
    'Aug',
    'Sep',
    'Oct',
    'Nov',
    'Dec',
]
colors = [
    '#604791',
    '#1d6b99',
    '#39a8a7',
    '#0f8755',
    '#76b349',
    '#f0af07',
    '#e37d05',
    '#cf513e',
    '#96356f',
    '#724173',
    '#9c4f97',
    '#696969',
]
title = "Average Monthly Temperature by Ecoregion"
xlabel = "Ecoregion"
ylabel = "Temperature"

In [62]:
options = {
    "labels": labels,
    "colors": colors,
    "title": title,
    "xlabel": xlabel,
    "ylabel": ylabel,
    "legend_location": "top-left",
    "height": "500px",
}

In [65]:
chart.feature_byFeature(features, xProperty, yProperties, **options)

# Andrea method